In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option('display.max_columns', 100)

PROJECT_ROOT = Path.cwd().parent

APPLIC_PATH = PROJECT_ROOT / 'data/raw/application_train.csv'
PROCESSED_PATH = PROJECT_ROOT / 'data/processed/application_features_baseline.csv'
BUREAU_PATH = PROJECT_ROOT / 'data/raw/bureau.csv'
BUREAU_BALANCE_PATH = PROJECT_ROOT / 'data/raw/bureau_balance.csv'

INTERIM_PATH = PROJECT_ROOT / 'data/interim'
INTERIM_BUREAU = INTERIM_PATH / 'bureau_agg.csv'

In [2]:
if not APPLIC_PATH.exists() or not BUREAU_PATH.exists() or not BUREAU_BALANCE_PATH.exists():
    raise FileNotFoundError(
        "Raw data not found. See README.md for download insturctions."
    )

df = pd.read_csv(APPLIC_PATH)

bureau_df = pd.read_csv(BUREAU_PATH)
bureau_balance_df = pd.read_csv(BUREAU_BALANCE_PATH)


Aggregate bureau balance table and merge it with bureau table by SK_ID_BUREAU key

In [3]:
bureau_balance_agg = bureau_balance_df.groupby('SK_ID_BUREAU').agg(
    loan_duration=('MONTHS_BALANCE', 'count'),
    prepaid_ratio=('STATUS', lambda x: (x == 'C').mean()),
    default_ever=('STATUS', lambda x: int((x == '5').any())),
    bad_dpd_ratio=('STATUS', lambda x: (x.isin(['2', '3', '4', '5']).mean())),
    bad_dpd_count=('STATUS', lambda x: (x.isin(['2', '3', '4', '5']).sum())),
    small_dpd_ratio=('STATUS', lambda x: (x == '1').mean()),
    small_dpd_count=('STATUS', lambda x: (x == '1').sum()),
    paid_in_time_ratio=('STATUS', lambda x: (x == '0').mean()),
    paid_in_time_count=('STATUS', lambda x: (x == '0').sum()),
    unknown_ratio=('STATUS', lambda x: (x == 'X').mean())
).reset_index()

In [4]:
bureau_df = bureau_df.merge(
    bureau_balance_agg, 
    how='left',
    on='SK_ID_BUREAU',
    )

Aggregate bureau table and merge it with main application table by SK_ID_CURR key. Engineering some features in bureau table before and after merging

In [5]:
bureau_df['early_closure_days'] = bureau_df['DAYS_CREDIT_ENDDATE'] - bureau_df['DAYS_ENDDATE_FACT']
bureau_df['credit_duration'] = abs(bureau_df['DAYS_CREDIT'] - bureau_df['DAYS_ENDDATE_FACT'])

bureau_df['cnt_curent_overdue'] = (
    (bureau_df['CREDIT_ACTIVE'] == 'Active') & 
    (bureau_df['CREDIT_DAY_OVERDUE'] > 0)
).astype(int)

bureau_df['overdue_days_active'] = np.where(
    bureau_df['CREDIT_ACTIVE'] == 'Active', 
    bureau_df['CREDIT_DAY_OVERDUE'],
    0
)

bureau_df['ever_overdue_flag'] = (bureau_df['AMT_CREDIT_MAX_OVERDUE'] > 0).astype(int)

bureau_df['overdue_ratio'] = np.where(
    bureau_df['AMT_CREDIT_SUM_OVERDUE'] > 0,
    bureau_df['AMT_CREDIT_SUM_OVERDUE'] / bureau_df['AMT_CREDIT_SUM'],
    0
)

bureau_df['credit_sum_active'] = np.where(
    bureau_df['CREDIT_ACTIVE'] == 'Active',
    bureau_df['AMT_CREDIT_SUM'],
    0
)

bureau_df['annuity_active'] = np.where(
    bureau_df['CREDIT_ACTIVE'] == 'Active',
    bureau_df['AMT_ANNUITY'],
    0
)

In [6]:
bureau_agg = bureau_df.groupby('SK_ID_CURR').agg(
    sold_times=('CREDIT_ACTIVE', lambda x: x.isin(['Sold', 'Bad debt']).sum()),
    closed_ratio=('CREDIT_ACTIVE', lambda x: (x == 'Closed').mean()),
    active_credits=('CREDIT_ACTIVE', lambda x: (x == 'Active').sum()),
    first_credit_time=('DAYS_CREDIT', 'min'),
    overdue_days_mean=('CREDIT_DAY_OVERDUE', 'mean'),
    overdue_days_active_mean=('overdue_days_active', 'mean'),
    overdue_active_max=('CREDIT_DAY_OVERDUE', 'max'),
    overdue_historical_max=('AMT_CREDIT_MAX_OVERDUE', 'max'),
    loans_ever_overdue=('ever_overdue_flag', 'sum'),
    loans_overdue_ratio=('ever_overdue_flag', 'mean'),
    overdue_credits_active=('cnt_curent_overdue', 'sum'),
    overdue_ammount_active=('AMT_CREDIT_SUM_OVERDUE', 'sum'),
    overdue_ratio_max=('overdue_ratio', 'max'),
    early_closure_days_ratio=('early_closure_days', 'mean'),
    credit_duration_mean=('credit_duration', 'mean'),
    prolonged_max=('CNT_CREDIT_PROLONG', 'max'),
    prolonged_times=('CNT_CREDIT_PROLONG', 'sum'),
    credit_sum_mean=('AMT_CREDIT_SUM', 'mean'),
    active_credit_sum=('credit_sum_active', 'sum'),
    debt_max=('AMT_CREDIT_SUM_DEBT', 'max'),
    debt_mean=('AMT_CREDIT_SUM_DEBT', 'mean'),
    credit_limit_max=('AMT_CREDIT_SUM_LIMIT', 'max'),
    has_credit_card=('CREDIT_TYPE', lambda x: int((x == 'Credit card').any())),
    credit_card_cnt=('CREDIT_TYPE', lambda x: (x == 'Credit card').sum()),
    low_risk_loans=('CREDIT_TYPE', lambda x: x.isin(['Car loan', 'Mortgage', 'Loan for business development']).sum()),
    has_microloan=('CREDIT_TYPE', lambda x: int((x == 'Microloan').any())),
    consumer_credit_sum=('CREDIT_TYPE', lambda x: (x == 'Consumer credit').sum()),
    last_credit_update=('DAYS_CREDIT_UPDATE', 'max'),
    first_credit_update=('DAYS_CREDIT_UPDATE', 'min'),
    current_annuity=('annuity_active', 'sum'),
    annuity_mean=('AMT_ANNUITY', 'mean'),
    loan_duration_avg=('loan_duration', 'mean'),
    loan_duration_max=('loan_duration', 'max'),
    prepaid_ratio_avg=('prepaid_ratio', 'mean'),
    defaults=('default_ever', 'sum'),
    worst_dpd=('bad_dpd_ratio', 'max'),
    bad_dpd_avg=('bad_dpd_ratio', 'mean'),
    bad_dpd_cnt=('bad_dpd_count', 'sum'),
    bad_dpd_times_max=('bad_dpd_count', 'max'),
    bigest_small_dpd=('small_dpd_ratio', 'max'),
    small_dpd_avg=('small_dpd_ratio', 'mean'),
    small_dpd_cnt=('small_dpd_count', 'sum'),
    paid_in_time_avg=('paid_in_time_ratio', 'mean'),
    paid_in_time_cnt=('paid_in_time_count', 'sum'),
    unkown_ratio_avg=('unknown_ratio', 'mean')
).reset_index()

Save aggregated table to interim folder to convinient reuse it in future

In [7]:
bureau_agg.to_csv(
    INTERIM_BUREAU,
    index=False
)

Load aggregated table from interim folder to save time due to long aggregation.

In [8]:
bureau_agg = pd.read_csv(INTERIM_BUREAU)

In [9]:
df = df.merge(
    bureau_agg,
    how='left',
    on='SK_ID_CURR'
)

In [10]:
df['debt_ratio_mean'] = np.where(
   df['credit_sum_mean'] > 0,
   df['debt_mean'] / df['credit_sum_mean'],
   0
)

Stop of bureau feature engineering

---

In [11]:
#365243 means NaN in dataset, we add DAYS_EMPLOYED_MISSING as borrowers whith NaN values have lower default rate
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].replace(365243, np.nan)

df['days_employed_missing'] = df['DAYS_EMPLOYED'].isna().astype(int)
df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].fillna(-1)

In [12]:
df['credit_income_ratio'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']
df['annuity_income_ratio'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

In [13]:
# smoothing skewed peaks of data
df['log_income'] = np.log1p(df['AMT_INCOME_TOTAL'])
df['log_credit'] = np.log1p(df['AMT_CREDIT'])

In [14]:
df['has_many_children'] = (df['CNT_CHILDREN'] >= 3).astype(int)

# include after baseline
# df['cnt_children_capped'] = df['CNT_CHILDREN'].clip(upper=3) 

In [15]:
df['ext_source_1_missing'] = df['EXT_SOURCE_1'].isna().astype(int)
df['ext_source_3_missing'] = df['EXT_SOURCE_3'].isna().astype(int)

df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].fillna(-1)

In [16]:
cat_features = df.select_dtypes('object').apply(pd.Series.nunique, axis=0)

for num, name in zip(cat_features, cat_features.index):
    if num <= 2:
        df[name] = df[name].astype('category').cat.codes.replace(-1, np.nan)

df = pd.get_dummies(df)

replacing binary categories to codes (0, 1), all NaN preserved. One-Hot encoding for all categorical features with more than 2 classes

In [17]:
df.drop(columns='SK_ID_CURR', inplace=True)

drop useless features

In [18]:
df.to_csv(
    PROCESSED_PATH,
    index=False
    )

In [19]:
test_initial_df = pd.read_csv(APPLIC_PATH)

In [20]:
test_agg_df = test_initial_df.merge(
    bureau_agg,
    how='left',
    on='SK_ID_CURR'
)

In [21]:
bureau_missing_sk_id = test_agg_df.loc[test_agg_df['AMT_REQ_CREDIT_BUREAU_QRT'].notna() & test_agg_df['sold_times'].isna(), 'SK_ID_CURR']
bureau_missing_sk_id.isin(bureau_agg['SK_ID_CURR']).sum()

np.int64(0)